In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam #optimizer of keras

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels) #shape of image
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5) #optimizer of gan

        # Build and compile the discriminator,only to keras
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))  #Input():用来实例化一个keras张量
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.

        X_train = np.expand_dims(X_train, axis=3)  #expand_dims用于扩充数组形状
        print(np.shape(X_train))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1

        if not os.path.isdir('images_gan'):
            os.mkdir('images_gan')
        fig.savefig("images_gan/%d.png" % epoch)
        plt.close()

    # def sample_images(self, epoch):
    #     r, c = 20, 20
    #     noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    #     gen_imgs = self.generator.predict(noise)
    #     # Rescale images 0 - 1
    #     gen_imgs = 0.5 * gen_imgs + 0.5
    #     fig1, axs1 = plt.subplots(r//2, c//2)
    #     cnt = 0
    #     for i in range(r//2):
    #         for j in range(c//2):
    #             axs1[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
    #             axs1[i, j].axis('off')
    #             cnt += 1
    #     fig2, axs2 = plt.subplots(r // 2, c // 2)
    #     for i in range(r // 2):
    #         for j in range(c // 2):
    #             axs2[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
    #             axs2[i, j].axis('off')
    #             cnt += 1
    #     if not os.path.isdir('images_gan'):
    #         os.mkdir('images_gan')
    #     fig1.savefig("images_gan/%d_1.png" % epoch)
    #     fig2.savefig("images_gan/%d_2.png" % epoch)
    #     plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30001, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.788099, acc.: 23.44%] [G loss: 0.612286]
1 [D loss: 0.464023, acc.: 69.53%] [G loss: 0.670945]
2 [D loss: 0.390120, acc.: 67.97%] [G loss: 0.805750]
3 [D loss: 0.312985, acc.: 85.94%] [G loss: 0.917845]
4 [D loss: 0.273126, acc.: 92.19%] [G loss: 1.054156]
5 [D loss: 0.228314, acc.: 96.09%] [G loss: 1.189910]
6 [D loss: 0.217519, acc.: 97.66%] [G loss: 1.319785]
7 [D loss: 0.169068, acc.: 100.00%] [G loss: 1.432082]
8 [D loss: 0.166176, acc.: 99.22%] [G loss: 1.478143]
9 [D loss: 0.148547, acc.: 100.00%] [G loss: 1.570642]
10 [D loss: 0.141158, acc.: 100.00%] [G loss: 1.720544]
11 [D loss: 0.128138, acc.: 100.00%] [G loss: 1.798405]
12 [D loss: 0.123029, acc.: 100.00%] [G loss: 1.841101]
13 [D loss: 0.099491, acc.: 100.00%] [G loss: 2.012278]
14 [D loss: 0.093234, acc.: 100.00%] [G loss: 1.983160]
15 [D loss: 0.082076, acc.: 100.00%] [G loss: 2.144734]
16 [D loss: 0.076909, acc.: 100.00%] [G loss: 2.197794]
17 [D loss: 0.072698, acc.: 100.00%] [G loss: 2.201931]
18 [D loss

152 [D loss: 0.149360, acc.: 96.88%] [G loss: 2.469944]
153 [D loss: 0.228243, acc.: 95.31%] [G loss: 2.207466]
154 [D loss: 0.252548, acc.: 89.84%] [G loss: 2.840439]
155 [D loss: 0.367540, acc.: 82.03%] [G loss: 2.564150]
156 [D loss: 0.230989, acc.: 93.75%] [G loss: 2.725098]
157 [D loss: 0.416397, acc.: 77.34%] [G loss: 2.696660]
158 [D loss: 0.380055, acc.: 81.25%] [G loss: 2.530784]
159 [D loss: 0.338089, acc.: 84.38%] [G loss: 2.592706]
160 [D loss: 0.298266, acc.: 89.06%] [G loss: 2.282852]
161 [D loss: 0.245759, acc.: 92.19%] [G loss: 2.919836]
162 [D loss: 0.556242, acc.: 71.09%] [G loss: 2.501062]
163 [D loss: 0.431772, acc.: 75.00%] [G loss: 2.681445]
164 [D loss: 0.701291, acc.: 61.72%] [G loss: 2.160986]
165 [D loss: 0.194478, acc.: 96.88%] [G loss: 2.733213]
166 [D loss: 0.521388, acc.: 75.00%] [G loss: 2.239455]
167 [D loss: 0.259730, acc.: 90.62%] [G loss: 3.082663]
168 [D loss: 1.019264, acc.: 49.22%] [G loss: 1.614970]
169 [D loss: 0.383789, acc.: 75.00%] [G loss: 1.

...

29369 [D loss: 0.623198, acc.: 60.16%] [G loss: 0.989041]
29370 [D loss: 0.664024, acc.: 61.72%] [G loss: 1.027732]
29371 [D loss: 0.604780, acc.: 72.66%] [G loss: 0.965367]
29372 [D loss: 0.589042, acc.: 74.22%] [G loss: 0.946486]
29373 [D loss: 0.701819, acc.: 51.56%] [G loss: 0.950722]
29374 [D loss: 0.676445, acc.: 57.03%] [G loss: 0.959879]
29375 [D loss: 0.590653, acc.: 67.97%] [G loss: 0.939867]
29376 [D loss: 0.625783, acc.: 64.06%] [G loss: 0.918999]
29377 [D loss: 0.660980, acc.: 61.72%] [G loss: 0.906709]
29378 [D loss: 0.701507, acc.: 53.91%] [G loss: 0.918153]
29379 [D loss: 0.660133, acc.: 62.50%] [G loss: 0.936971]
29380 [D loss: 0.664722, acc.: 61.72%] [G loss: 0.944201]
29381 [D loss: 0.654221, acc.: 58.59%] [G loss: 0.983276]
29382 [D loss: 0.631824, acc.: 65.62%] [G loss: 0.975762]
29383 [D loss: 0.637390, acc.: 58.59%] [G loss: 0.946422]
29384 [D loss: 0.611310, acc.: 64.84%] [G loss: 0.949205]
29385 [D loss: 0.667899, acc.: 64.84%] [G loss: 0.931540]
29386 [D loss:

29513 [D loss: 0.643387, acc.: 64.06%] [G loss: 0.945539]
29514 [D loss: 0.649670, acc.: 61.72%] [G loss: 0.889680]
29515 [D loss: 0.655376, acc.: 60.94%] [G loss: 1.001803]
29516 [D loss: 0.615776, acc.: 66.41%] [G loss: 0.919517]
29517 [D loss: 0.622578, acc.: 60.16%] [G loss: 0.979890]
29518 [D loss: 0.641115, acc.: 62.50%] [G loss: 0.967993]
29519 [D loss: 0.642607, acc.: 60.16%] [G loss: 0.868176]
29520 [D loss: 0.644760, acc.: 66.41%] [G loss: 0.954414]
29521 [D loss: 0.659587, acc.: 60.94%] [G loss: 0.981138]
29522 [D loss: 0.599943, acc.: 67.97%] [G loss: 0.976313]
29523 [D loss: 0.651573, acc.: 62.50%] [G loss: 0.948144]
29524 [D loss: 0.620969, acc.: 65.62%] [G loss: 0.929458]
29525 [D loss: 0.639468, acc.: 53.91%] [G loss: 0.943641]
29526 [D loss: 0.625377, acc.: 60.94%] [G loss: 0.909568]
29527 [D loss: 0.706494, acc.: 58.59%] [G loss: 0.895576]
29528 [D loss: 0.708919, acc.: 54.69%] [G loss: 1.025236]
29529 [D loss: 0.683932, acc.: 57.81%] [G loss: 0.917772]
29530 [D loss:

29657 [D loss: 0.645334, acc.: 57.81%] [G loss: 0.953618]
29658 [D loss: 0.604371, acc.: 73.44%] [G loss: 0.969140]
29659 [D loss: 0.666201, acc.: 61.72%] [G loss: 1.005483]
29660 [D loss: 0.650842, acc.: 61.72%] [G loss: 0.988652]
29661 [D loss: 0.614387, acc.: 66.41%] [G loss: 0.998205]
29662 [D loss: 0.658784, acc.: 58.59%] [G loss: 1.027919]
29663 [D loss: 0.648753, acc.: 62.50%] [G loss: 1.015289]
29664 [D loss: 0.674011, acc.: 58.59%] [G loss: 1.035577]
29665 [D loss: 0.621269, acc.: 62.50%] [G loss: 0.961860]
29666 [D loss: 0.615479, acc.: 62.50%] [G loss: 0.912952]
29667 [D loss: 0.715957, acc.: 51.56%] [G loss: 0.870108]
29668 [D loss: 0.635967, acc.: 67.97%] [G loss: 0.941435]
29669 [D loss: 0.668531, acc.: 61.72%] [G loss: 0.960012]
29670 [D loss: 0.643032, acc.: 64.06%] [G loss: 0.922682]
29671 [D loss: 0.636264, acc.: 62.50%] [G loss: 0.899930]
29672 [D loss: 0.679097, acc.: 55.47%] [G loss: 0.952876]
29673 [D loss: 0.661596, acc.: 60.94%] [G loss: 0.941914]
29674 [D loss:

29801 [D loss: 0.659608, acc.: 57.03%] [G loss: 0.941451]
29802 [D loss: 0.633747, acc.: 67.97%] [G loss: 0.874270]
29803 [D loss: 0.656436, acc.: 54.69%] [G loss: 0.864670]
29804 [D loss: 0.654397, acc.: 57.81%] [G loss: 0.899214]
29805 [D loss: 0.639065, acc.: 63.28%] [G loss: 0.918900]
29806 [D loss: 0.627302, acc.: 64.06%] [G loss: 1.010650]
29807 [D loss: 0.633377, acc.: 63.28%] [G loss: 1.065701]
29808 [D loss: 0.622648, acc.: 64.84%] [G loss: 1.017492]
29809 [D loss: 0.566364, acc.: 71.88%] [G loss: 0.988267]
29810 [D loss: 0.637371, acc.: 57.81%] [G loss: 0.951801]
29811 [D loss: 0.612555, acc.: 64.06%] [G loss: 0.954390]
29812 [D loss: 0.639507, acc.: 58.59%] [G loss: 0.949507]
29813 [D loss: 0.660958, acc.: 60.16%] [G loss: 0.871093]
29814 [D loss: 0.649036, acc.: 65.62%] [G loss: 0.931620]
29815 [D loss: 0.611117, acc.: 68.75%] [G loss: 0.950589]
29816 [D loss: 0.659570, acc.: 60.94%] [G loss: 0.890790]
29817 [D loss: 0.644358, acc.: 62.50%] [G loss: 0.963827]
29818 [D loss:

29947 [D loss: 0.640615, acc.: 64.06%] [G loss: 0.980902]
29948 [D loss: 0.677960, acc.: 54.69%] [G loss: 0.872909]
29949 [D loss: 0.650069, acc.: 65.62%] [G loss: 0.942634]
29950 [D loss: 0.636501, acc.: 65.62%] [G loss: 0.910490]
29951 [D loss: 0.646591, acc.: 59.38%] [G loss: 0.984769]
29952 [D loss: 0.647128, acc.: 64.06%] [G loss: 0.919590]
29953 [D loss: 0.637625, acc.: 63.28%] [G loss: 0.922016]
29954 [D loss: 0.622737, acc.: 69.53%] [G loss: 0.968631]
29955 [D loss: 0.644314, acc.: 60.94%] [G loss: 0.941863]
29956 [D loss: 0.680029, acc.: 53.91%] [G loss: 0.983010]
29957 [D loss: 0.615440, acc.: 66.41%] [G loss: 0.987896]
29958 [D loss: 0.615407, acc.: 65.62%] [G loss: 0.915720]
29959 [D loss: 0.712522, acc.: 60.94%] [G loss: 0.948137]
29960 [D loss: 0.662176, acc.: 64.84%] [G loss: 1.031250]
29961 [D loss: 0.653114, acc.: 60.16%] [G loss: 0.965302]
29962 [D loss: 0.636556, acc.: 63.28%] [G loss: 0.959875]
29963 [D loss: 0.694273, acc.: 53.91%] [G loss: 0.829147]
29964 [D loss: